라이브러리 설치  
pip install transformers torch Pillow scipy  
테스트용  
  
TODO  
 label DB에서 추출해서 사용  
 이미지 받는거 변수로 해야함  
 GPU사용 확인  
 찾은 사물 현재 파이썬 set으로 사용 파일로 넘길지 이 노트북에서 끝까지 처리할지 정해야함.

In [ ]:
from PIL import Image
import torch
from transformers import pipeline

if torch.cuda.is_available():
    print(f"GPU ({torch.cuda.get_device_name(0)})")
    device_option = 0
else:
    print("CPU")
    device_option = -1

# 'zero-shot-object-detection' 파이프라인 불러오기
#    - model: 사용할 모델을 지정합니다. 'google/owlv2-base-patch16-ensemble'은 성능이 좋은 최신 모델입니다.
#    - task: 수행할 작업을 지정합니다.
detector = pipeline(model="google/owlv2-base-patch16-ensemble", task="zero-shot-object-detection", device=device_option)
print("모델 로딩 완료.")

try:
    #image_path = "raw image/1930.jpg" #에어컨, 리볼버? 피스톨?, LED등
    image_path = "raw image/1400.jpg" #감자
    image = Image.open(image_path)
    print(f"'{image_path}' 이미지를 성공적으로 불러왔습니다.")

    # 이미지에서 찾고 싶은 객체 목록 정의
    #    - 이 목록을 수정하여 원하는 객체를 찾을 수 있습니다.
    #    - "a photo of a cat" 처럼 구체적으로 작성하면 정확도가 올라갑니다.
    candidate_labels = ["air conditioner", "revolver", "pistol", "LED light", "candle", "potato"]
    print(f"탐색할 객체: {candidate_labels}")

    # 모델 실행하여 객체 탐지
    #     - 이미지와 객체 후보 목록을 모델에 전달합니다.
    print("객체 탐지를 시작합니다...")
    predictions = detector(image, candidate_labels=candidate_labels)
    print("객체 탐지 완료.")

    # 결과 출력
    #    - 모델이 찾은 각 객체의 정보를 보기 쉽게 출력합니다.
    print("\n--- 탐지 결과 ---")
    if not predictions:
        print("이미지에서 지정된 객체를 찾지 못했습니다.")
    else:
        labels = []
        for prediction in predictions:
            labels.append(prediction["label"])
            
            # 신뢰도 점수가 0.1 이상인 결과만 출력 (불필요한 결과 제거)
            if prediction["score"] > 0.1:
                labels.append(prediction["label"])
        set_labels = set(labels)  # 중복 제거
        print(set_labels)

except FileNotFoundError:
    print(f"오류: '{image_path}' 파일을 찾을 수 없습니다. 코드와 같은 폴더에 이미지 파일이 있는지 확인하세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

CPU


Device set to use cpu


모델 로딩 완료.
'raw image/1400.jpg' 이미지를 성공적으로 불러왔습니다.
탐색할 객체: ['air conditioner', 'revolver', 'pistol', 'LED light', 'candle', 'potato']
객체 탐지를 시작합니다...
객체 탐지 완료.

--- 탐지 결과 ---
{'potato'}
